In [ ]:
#!pip install openai
import os
import pdb
import time
import openai
import requests
import pandas as pd 

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, f1_score

In [4]:
# change this key frequently

OPENAI_API_KEY = ""

In [5]:
openai.api_key = OPENAI_API_KEY

In [6]:
def label_message(meta_statement, definition, article, prompt, role_user="user"):

    completion = openai.chat.completions.create(model="gpt-4",
                                              temperature = 0, # make sure that all answers are the same, 
                                              max_tokens = 50, # 8,000 is the max token length for GPT3.5
                                              messages=[{"role": "system", "content": "You are a news classifier."},
                                                        {"role": role_user, "content": meta_statement},
                                                        # {"role": role_user, "content": f'Read this definition {definition}'},
                                                        {"role": role_user, "content": f'Read this article {article}'},
                                                        {"role": role_user, "content": prompt}]) 
    return completion.choices[0].message

In [7]:
news_articles = pd.read_json("old_data_clean_wo_missed_label.json")

In [9]:
def map_values(value):
    if value == 'irrelevant':
        return '5'
    elif value == 'relchanges':
        return '3'
    elif value == 'explicit':
        return '1'
    elif value == 'releconomy':
        return '4'
    elif value == 'implicit':
        return '2'
    else:
        return 'Unknown'

In [10]:
# Apply the function to create the new column
news_articles['label'] = news_articles['five_code'].apply(map_values)
news_articles['label'].unique()

array(['5', '3', '1', '4', '2'], dtype=object)

In [11]:
rel_news_articles = news_articles[news_articles['label'] != '5'].reset_index(drop=True)
len(rel_news_articles)

786

In [12]:
rel_news_articles['label'].unique()

array(['3', '1', '4', '2'], dtype=object)

In [13]:
inequality_def = """Criteria for referencing economic inequality in the United States:
(1) Wage, earnings, pay, and income inequality. 
(2) Causes and policy solutions related to economic inequality. 
(3) Relational or comparative language among different social class groups."""

In [14]:
research_goal = """We categorize articles that are related to issues of income inequality, changes in income or wealth, general economic conditions."""

In [17]:
cols = rel_news_articles.columns.to_list()

In [18]:
keep_cols = cols[:37] 
keep_cols.append('token_count')
#keep_cols

In [19]:
# only binary 1: 
binary1 = rel_news_articles[keep_cols]

In [ ]:
messages = []
max_char = 8000*3 
max_token = 7500 # cushion for definition and meta-statement
for i in range(len(binary1)):
    if i % 100 == 0: 
        print("processing message", i)
    title = binary1.loc[i, 'title']
    content = binary1.loc[i, 'text']
    if binary1.loc[i,'token_count'] <= max_token: 
        content = f'title: {title} \n article: {content}'
    else: 
        content = f'title: {title} \n article: {content[:max_char]}'
    article_prompt = "Does the article reference American economic inequality? Respond with 'Yes' if article meets any or all criteria referencing American economic inequality and 'No' if article meets none of the criteria, and explain why in 1 sentence."
    while True: 
        try: 
            output_message = label_message(meta_statement=research_goal, 
                                           article=content, 
                                           definition=inequality_def, 
                                           prompt=article_prompt)
            break 
        except Exception as e:
             time.sleep(30)       # add time lag of 30 seconds when an error occurs 
    messages.append(output_message) 
#     pdb.set_trace()

In [23]:
binary1['output_message'] = [message.content for message in messages]

In [24]:
binary1['output_message'] = binary1['output_message'].str.lower()

In [26]:
def parse_answers(binary1, messages): 
    '''
    parse the answer to binary no - non-inequality, yes - inequality 
    '''
    predicted = []
    for message in messages: 
        if message.startswith("yes"): 
            predicted.append('yes')
        elif message.startswith("no"): 
            predicted.append('no')
        else: 
            predicted.append('flag')
    binary1['predicted_label'] = predicted
    return 

In [27]:
parse_answers(binary1, binary1['output_message'])

In [31]:
# if code in (1, 2) then it's inequality 
labels = [int(code_i in (1, 2)) for code_i in binary1.code]

In [33]:
binary1['label'] = labels

In [36]:
# assign all predicted labels as 0 
binary1['pred'] = [0]*len(binary1)

In [37]:
# change those predicted as inequality to 1 
binary1.loc[binary1.predicted_label=='yes', 'pred'] = 1

In [43]:
# save the output for future comparison if needed 
binary1.to_csv("../data/inequality_dataset_GPT_Inequality_NoDefinition.csv", index=False)